<a href="https://colab.research.google.com/github/Vlasovets/nlp_adversarial_examples/blob/master/LogReg_success_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir data

In [3]:
%cd data

/content/data


In [0]:
!mv /content/ag_news_csv.zip /content/data

In [5]:
!unzip ag_news_csv.zip

Archive:  ag_news_csv.zip
 extracting: ag_news_csv/classes.txt  
  inflating: ag_news_csv/readme.txt  
  inflating: ag_news_csv/test.csv    
  inflating: ag_news_csv/train.csv   


In [5]:
%cd /content/data/ag_news_csv

/content/data/ag_news_csv


In [0]:
import pandas as pd
import numpy as np

from sklearn import model_selection, linear_model, metrics
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [0]:
!jupyter nbextension enable --py widgetsnbextension

In [7]:
%cd /Users/xvlasol/Desktop/samples/ag_news_csv

C:\Users\xvlasol\Desktop\samples\ag_news_csv


#Data preparation

##AG News dataset
The training dataset will be the same for every model

In [8]:
train = pd.read_csv('train.csv', header=None)
train.columns = ['Class','Title', 'Description']

test_ag = pd.read_csv('test.csv', header=None)
test_ag.columns = ['Class','Title', 'Description']

print(train.head(), test_ag.head())

   Class                                              Title  \
0      3  Wall St. Bears Claw Back Into the Black (Reuters)   
1      3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2      3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3      3  Iraq Halts Oil Exports from Main Southern Pipe...   
4      3  Oil prices soar to all-time record, posing new...   

                                         Description  
0  Reuters - Short-sellers, Wall Street's dwindli...  
1  Reuters - Private investment firm Carlyle Grou...  
2  Reuters - Soaring crude prices plus worries\ab...  
3  Reuters - Authorities have halted oil export\f...  
4  AFP - Tearaway world oil prices, toppling reco...      Class                                              Title  \
0      3                  Fears for T N pension after talks   
1      4  The Race is On: Second Private Team Sets Launc...   
2      4      Ky. Company Wins Grant to Study Peptides (AP)   
3      4      Prediction Unit Helps For

In [0]:
test_ag = test_ag[:987]

In [10]:
y_train = train['Class']
y_test = test_ag['Class']

print(len(y_train), len(y_test))

120000 987


##DeepWordBug Samples

Not possible to get y_test

In [0]:
test_dwb = pd.read_csv('DWB_output.txt', sep='\n', header=None)
test_dwb.columns = ['Description']

#print(train_ag.head(), test_ag.head())
test_dwb.head()

,Description
0,unctad optimistic on global fdi inflows the un...
1,hundreds mourn loss of student killed by polic...
2,rebels attack in central iraq and the north a ...
3,no sign yet of predicted big california earthq...
4,accounting body delays rule on expensing optio...


##CNN Samples

In [11]:
test_cnn = pd.read_csv('PWWS_cnn_adv_7600.txt', sep='\n', header=None)
test_cnn.columns = ['Description']

test_cnn.head()

,Description
0,Fears for T N pension after talks . Unions rep...
1,The Race is On : Second Private Team Sets Laun...
2,Ky. Company Wins President to learn Peptides (...
3,Prediction Unit Helps Forecast Wildfires (AP)....
4,Calif. Aims to Limit Farm-Related Smog (AP). A...


Remove the end of each line containing unnecessary statisctics like 'sub_rate: 0.034482758620689655; NE_rate: 0.0'

In [12]:
test_cnn['new_Description'] = test_cnn['Description'].str.split('sub_rate').str[0]
del test_cnn['Description']
test_cnn.columns = ['Description']

test_cnn.head()

,Description
0,Fears for T N pension after talks . Unions rep...
1,The Race is On : Second Private Team Sets Laun...
2,Ky. Company Wins President to learn Peptides (...
3,Prediction Unit Helps Forecast Wildfires (AP)....
4,Calif. Aims to Limit Farm-Related Smog (AP). A...


In [15]:
len(test_cnn)

987

In [0]:
test_cnn = test_cnn[:987]

##LSTM Samples



In [16]:
test_lstm = pd.read_csv('LSTM_1000.txt', sep='\n', header=None)
test_lstm.columns = ['Description']

test_lstm.head()

,Description
0,Fears for T N pension after talks . Unions rep...
1,The Race is On : Second Private Team Sets Laun...
2,Ky. Company Wins Grant to take Peptides ( AFP ...
3,Prediction Unit Helps Forecast Wildfires (AP)....
4,Calif. Aims to Limit Farm - Related Smog ( AP ...


In [17]:
test_lstm['new_Description'] = test_lstm['Description'].str.split('sub_rate').str[0]
del test_lstm['Description']
test_lstm.columns = ['Description']

test_lstm.head()

,Description
0,Fears for T N pension after talks . Unions rep...
1,The Race is On : Second Private Team Sets Laun...
2,Ky. Company Wins Grant to take Peptides ( AFP ...
3,Prediction Unit Helps Forecast Wildfires (AP)....
4,Calif. Aims to Limit Farm - Related Smog ( AP ...


Make all sample of the same size.

In [0]:
test_lstm = test_lstm[:987]

In [19]:
len(test_lstm)

987

##BiLSTM Samples

In [20]:
test_bilstm = pd.read_csv('BiLSTM_978.txt', sep='\n', header=None)
test_bilstm.columns = ['Description']

test_bilstm.head()

,Description
0,Fears for T N pension after talks . Unions rep...
1,The Race is On : Second individual Team Sets L...
2,Ky. Company Wins Grant to consider Peptides ( ...
3,Prediction Unit Helps Forecast Wildfires ( AP ...
4,Calif. Aims to Limit Farm - Related Smog ( AP ...


In [21]:
test_bilstm['new_Description'] = test_bilstm['Description'].str.split('sub_rate').str[0]
del test_bilstm['Description']
test_bilstm.columns = ['Description']

test_bilstm.head()

,Description
0,Fears for T N pension after talks . Unions rep...
1,The Race is On : Second individual Team Sets L...
2,Ky. Company Wins Grant to consider Peptides ( ...
3,Prediction Unit Helps Forecast Wildfires ( AP ...
4,Calif. Aims to Limit Farm - Related Smog ( AP ...


In [22]:
len(test_bilstm)

987

#Vectorizing

##AG news dataset

In [23]:
def tokenizersplit(str):
    return str.split();

tfidf = TfidfVectorizer(tokenizer=tokenizersplit, encoding='utf-8', min_df=2, ngram_range=(1, 2), max_features=25000)
tfidf.fit(train['Description'])

X_train = tfidf.transform(train['Description'])
X_test_ag = tfidf.transform(test_ag['Description'])

C:\Users\xvlasol\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


##CNN sample

In [0]:
X_test_cnn = tfidf.transform(test_cnn['Description'])

##LSTM sample

In [0]:
X_test_lstm = tfidf.transform(test_lstm['Description'])

##BiLSTM sample

In [0]:
X_test_bilstm = tfidf.transform(test_bilstm['Description'])

# Logistic regression model

##AG news dataset

In [27]:
model = linear_model.LogisticRegression(verbose = 1)
model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\xvlasol\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.2s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [28]:
y_pred_ag = model.predict(X_test_ag)

print(metrics.classification_report(np.array(y_test), y_pred_ag, labels=[1, 2, 3, 4]))

              precision    recall  f1-score   support

           1       0.93      0.86      0.90       264
           2       0.93      0.97      0.95       270
           3       0.79      0.87      0.83       204
           4       0.89      0.84      0.86       249

    accuracy                           0.89       987
   macro avg       0.88      0.89      0.88       987
weighted avg       0.89      0.89      0.89       987



##CNN Samples

In [29]:
y_pred_cnn = model.predict(X_test_cnn)

print(metrics.classification_report(np.array(y_test), y_pred_cnn, labels=[1, 2, 3, 4]))

              precision    recall  f1-score   support

           1       0.27      0.38      0.32       264
           2       0.36      0.32      0.34       270
           3       0.23      0.18      0.20       204
           4       0.27      0.23      0.25       249

    accuracy                           0.29       987
   macro avg       0.28      0.28      0.28       987
weighted avg       0.29      0.29      0.28       987



##LSTM Samples

In [30]:
y_pred_lstm = model.predict(X_test_lstm)

print(metrics.classification_report(np.array(y_test), y_pred_lstm, labels=[1, 2, 3, 4]))

              precision    recall  f1-score   support

           1       0.49      0.70      0.58       264
           2       0.79      0.52      0.62       270
           3       0.70      0.64      0.67       204
           4       0.59      0.57      0.58       249

    accuracy                           0.61       987
   macro avg       0.64      0.61      0.61       987
weighted avg       0.64      0.61      0.61       987



##BiLSTM Samples

In [31]:
y_pred_bilstm = model.predict(X_test_bilstm)

print(metrics.classification_report(np.array(y_test), y_pred_bilstm, labels=[1, 2, 3, 4]))

              precision    recall  f1-score   support

           1       0.59      0.69      0.64       264
           2       0.85      0.61      0.71       270
           3       0.73      0.72      0.72       204
           4       0.65      0.73      0.69       249

    accuracy                           0.68       987
   macro avg       0.70      0.69      0.69       987
weighted avg       0.70      0.68      0.69       987



LR is trained on CNN samples show the worst result of the classification which means that AE technique based on CNN was the most succesful.

#Random forest model

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, n_jobs=-1, verbose = 2)
# Train the model on training data
rf.fit(X_train, y_train);

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100building tree 5 of 100

building tree 6 of 100
building tree 7 of 100
building tree 8 of 100


In [0]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

print(metrics.classification_report(np.array(y_test), predictions, labels=[1, 2, 3, 4]))

In [0]:
from sklearn import svm, grid_search
def svc_param_selection(X, y, nfolds):
    Cs = [0.01, 1, 10]
    gammas = [0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [0]:
svc_param_selection(X_train, y_train, 10)